# **Homework - #07**

**Prof. Jonathan Russert**

**Name: Mohamed Aarif Mohamed Sulaiman**

**PFW ID: 900405565**

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
from huggingface_hub import login

login()

In [5]:
gpt_model = AutoModelForCausalLM.from_pretrained("gpt2")
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")

meta_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
meta_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

In [7]:
gemma_model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it")
gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
smol_model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct")
smol_tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct")

In [11]:
def set_pad_token_if_needed(tokenizer):
  if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

set_pad_token_if_needed(gpt_tokenizer)
set_pad_token_if_needed(meta_tokenizer)
set_pad_token_if_needed(gemma_tokenizer)
set_pad_token_if_needed(smol_tokenizer)

In [13]:
def generate_text(model, tokenizer, prompt, decoding_method="greedy", beams=3):
    inputs = tokenizer(prompt, return_tensors="pt", padding = True, truncation = True)

    attention_mask = inputs.get("attention_mask")
    if attention_mask is None:
        attention_mask = (inputs["input_ids"] != model.config.pad_token_id).long()

    if decoding_method == "greedy":
        # Greedy search
        output = model.generate(inputs["input_ids"], attention_mask=attention_mask, do_sample=False, max_length=50)
    elif decoding_method == "random":
        # Random sampling (multinomial sampling)
        output = model.generate(inputs["input_ids"], attention_mask=attention_mask, do_sample=True, max_length=50)
    elif decoding_method == "beam":
        # Beam search
        output = model.generate(inputs["input_ids"], attention_mask=attention_mask, num_beams=beams, max_length=50)
    elif decoding_method == "beam_random":
        # Beam search with random sampling
        output = model.generate(inputs["input_ids"], attention_mask=attention_mask, num_beams=beams, do_sample=True, max_length=50)

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [15]:
prompts = [
    "Once upon a time, there lived a King named Julius Caesar",
    "This movie was a good example of",
    "A farmer has 17 sheep, and all but 9 run away. How many are left?",
    "The future of artificial intelligence is"
]

# Decoding methods
methods = ["greedy", "random", "beam", "beam_random"]

In [17]:
results = {}

for prompt in prompts:
    for method in methods:
        results[f"GPT2_{prompt[:10]}_{method}"] = generate_text(gpt_model, gpt_tokenizer, prompt, decoding_method=method)
        results[f"Meta-Llama_{prompt[:10]}_{method}"] = generate_text(meta_model, meta_tokenizer, prompt, decoding_method=method)
        results[f"SMOL_{prompt[:10]}_{method}"] = generate_text(smol_model, smol_tokenizer, prompt, decoding_method=method)
        results[f"Google-Gemma_{prompt[:10]}_{method}"] = generate_text(gemma_model, gemma_tokenizer, prompt, decoding_method=method)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
C:\Users\moham09\AppData\Local\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\moham09\AppData\Local\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Sett

In [21]:
# Print results for comparison
with open("C:/Users/moham09/Downloads/text_results_hw7.txt","w") as file:
    for key, value in results.items():
        file.write(f"{key}:\n{value}\n\n")